In [ ]:
from torchvision.datasets import VOCDetection
from torch.utils.data import Dataset
import torch
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import ToTensor
from tqdm import tqdm
import numpy as np
import cv2 as cv

from dataset import YoloVOCDataset

In [ ]:
IMG_SIZE=416

voc_ds=VOCDetection(root='data',year='2012',image_set='train',download=False)
classdict=set()
anchor_boxex=[]
for _,label in tqdm(voc_ds, desc="数据集处理中"):
    for obj in label['annotation']['object']:
        classdict.add(obj['name'])
        xmin,ymin=int(obj['bndbox']['xmin']),int(obj['bndbox']['ymin'])
        xmax,ymax=int(obj['bndbox']['xmax']),int(obj['bndbox']['ymax'])
        w=xmax-xmin
        h=ymax-ymin
        anchor_boxex.append([w,h])
names=sorted(list(classdict))
id2name={i:c for i,c in enumerate(names)}
name2id={c:i for i,c in id2name.items()}

In [266]:
out=torch.randn((9,13,13,125))

# 假设参数
B = out.shape[0]    # batch size
S = 13          # 网格尺寸
A = 5  # anchors 数量
C = 20          # 类别数
anchors_tensor=torch.as_tensor([[1,1],[2,2],[3,3],[4,4],[5,5]])

In [256]:
# 将 out reshape 为 (B, S, S, A, 5+C)
out = out.view(B, S, S, A, 5 + C)

In [257]:
out.shape

torch.Size([2, 13, 13, 5, 25])

In [ ]:
pred = out[..., :5]

In [ ]:
pred.shape

In [ ]:
grid_x = torch.arange(S, device=out.device).view(1, S, 1, 1)  # 对应 row（Y）
grid_y = torch.arange(S, device=out.device).view(1, 1, S, 1)  # 对应 col（X）

In [ ]:
grid_x.shape,grid_y.shape

In [252]:
x = torch.sigmoid(pred[..., 0]) + grid_x
y = torch.sigmoid(pred[..., 1]) + grid_y

anchor_w = anchors_tensor[:, 0].view(1, 1, 1, A)  # shape: (1,1,1,A)
anchor_h = anchors_tensor[:, 1].view(1, 1, 1, A)  # shape: (1,1,1,A)
w = torch.exp(pred[..., 2]) * anchor_w
h = torch.exp(pred[..., 3]) * anchor_h

c = torch.sigmoid(pred[..., 4])

# 将变换后的5个数合并回去，得到 shape (B, S, S, A, 5)
transformed = torch.stack([x, y, w, h, c], dim=-1)

In [258]:
out[..., :5] = transformed

In [262]:
out = out.view(B, S, S, A * (5 + C))

In [267]:
len(out)

9